In [1]:
import numpy as np
import pandas as pd

In [2]:
new_model = 'GaussianProcess'
jc = 'RF-JC'

In [3]:
data_frame = pd.read_csv("../data/MultiRegression/gaussianProcess.csv",index_col=0)
data_frame.head(10)

,data,tau_x_score,prediction_time,buckets_per_rank,algo
0,Authorship,0.935897,0.779086,3.160000,GaussianProcess
1,Authorship,0.937500,0.700813,3.125000,GaussianProcess
2,Authorship,0.932540,0.701579,3.200000,GaussianProcess
3,Authorship,0.920635,0.699481,3.142857,GaussianProcess
4,Authorship,0.940476,0.701659,3.294118,GaussianProcess
5,Authorship,0.858974,0.551200,2.866667,RF-JC
6,Authorship,0.898810,0.536230,2.923077,RF-JC
7,Authorship,0.894841,0.539481,2.857143,RF-JC
8,Authorship,0.865079,0.541508,2.705882,RF-JC
9,Authorship,0.893849,0.538352,2.857143,RF-JC


In [4]:
v = data_frame.groupby(by=['algo','data']).apply(lambda x:np.std(x,axis=0)/np.sqrt(5)*1.96)
v

tau_x_score  prediction_time  buckets_per_rank
algo            data                                                      
GaussianProcess Authorship     0.006032         0.027428          0.052783
                IRIS           0.023374         0.008466          0.112311
                WINE           0.030404         0.001411          0.078212
                YEAST          0.004570         0.004273          0.027883
RF-JC           Authorship     0.014688         0.004567          0.063413
                IRIS           0.034296         0.001295          0.067223
                WINE           0.024321         0.001288          0.023856
                YEAST          0.005489         0.013415          0.029220

In [5]:
top_values = data_frame.groupby(by=['algo','data']).apply(lambda x:np.mean(x,axis=0) + np.std(x,axis=0)/np.sqrt(5)*1.96)
top_values


tau_x_score  prediction_time  buckets_per_rank
algo            data                                                      
GaussianProcess Authorship     0.939442         0.743951          3.237178
                IRIS           0.981152         0.198203          2.964692
                WINE           0.903314         0.229641          2.562339
                YEAST          0.814798         0.998719          5.293296
RF-JC           Authorship     0.896998         0.545921          2.905395
                IRIS           0.967629         0.351934          2.769604
                WINE           0.901094         0.356757          2.490523
                YEAST          0.817436         1.605028          4.796215

In [6]:
bottom_values = data_frame.groupby(by=['algo','data']).apply(lambda x:np.mean(x,axis=0) - np.std(x,axis=0)/np.sqrt(5)*1.96)
bottom_values

tau_x_score  prediction_time  buckets_per_rank
algo            data                                                      
GaussianProcess Authorship     0.927377         0.689096          3.131612
                IRIS           0.934403         0.181270          2.740070
                WINE           0.842506         0.226820          2.405915
                YEAST          0.805659         0.990173          5.237530
RF-JC           Authorship     0.867623         0.536787          2.778570
                IRIS           0.899038         0.349344          2.635158
                WINE           0.852451         0.354181          2.442810
                YEAST          0.806457         1.578199          4.737775

In [7]:
algo_names,data_sets= np.array([*bottom_values.index]).T
np.unique(algo_names)

array(['GaussianProcess', 'RF-JC'], dtype='<U15')

In [8]:
#algo1,algo2 = np.unique(algo_names)

data_sets = np.unique(data_sets)

In [9]:
significant_better = 0
overlapping = 0
for data in data_sets:
    new_model_bottom_tau,jc_bottom_tau = bottom_values.loc[(new_model,data),'tau_x_score'],bottom_values.loc[(jc,data),'tau_x_score']
    new_model_top_tau,jc_top_tau = bottom_values.loc[(new_model,data),'tau_x_score'],bottom_values.loc[(jc,data),'tau_x_score']
    print("DATA: ", data)
    print(f"{new_model}: ", new_model_bottom_tau, new_model_top_tau)
    print("JC: ", jc_bottom_tau, jc_top_tau)
    print("---")
    if jc_top_tau < new_model_bottom_tau:
        significant_better += 1
    if new_model_bottom_tau <= jc_top_tau and jc_bottom_tau <= new_model_top_tau:
        overlapping += 1
print("SIGINIFICANT BETTER: ", significant_better / len(data_sets))
print("OVERLAPPING: ", overlapping / len(data_sets))
print("WORSE: ", 1 - (significant_better / len(data_sets) + overlapping / len(data_sets)))


DATA:  Authorship
GaussianProcess:  0.9273773455342154 0.9273773455342154
JC:  0.8676232041293563 0.8676232041293563
---
DATA:  IRIS
GaussianProcess:  0.9344034138529799 0.9344034138529799
JC:  0.8990376545931931 0.8990376545931931
---
DATA:  WINE
GaussianProcess:  0.842506433170317 0.842506433170317
JC:  0.8524511933984603 0.8524511933984603
---
DATA:  YEAST
GaussianProcess:  0.8056589843113022 0.8056589843113022
JC:  0.8064570571024852 0.8064570571024852
---
SIGINIFICANT BETTER:  0.5
OVERLAPPING:  0.0
WORSE:  0.5
